In [ ]:
from ardac_utils import *
import matplotlib.pyplot as plt

In [ ]:
#use ardac helper functions to grab some fire zone polygons and output a geodataframe in 3338
#and get flammability data for those polygons
fz_gdf = get_area_gdf_by_category("fire_zones", 3338)
fz_data = get_data_for_gdf_polygons(fz_gdf, "flammability")

In [ ]:
#check out the data
fz_gdf.plot()

In [ ]:
fz_data.head()

In [ ]:
#subset the data for plotting:
#include just the 5-model avg, and just RCP 8.5
plot_subset = fz_data[(fz_data['model'] == '5modelAvg') & (fz_data['scenario'] == 'rcp85')]
plot_subset.head()

In [ ]:
#merge with the geodataframe using the area id
plot_gdf = fz_gdf.merge(plot_subset, how='left', on='id')

In [ ]:
#plot
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(20,14))

plot_gdf.boundary.plot(color='black', linewidth=0.1, ax=ax1)
plot_gdf[plot_gdf['date_range'] == '2010-2039'].plot(column='mean', cmap='Reds', ax=ax1)

plot_gdf.boundary.plot(color='black', linewidth=0.1, ax=ax2)
plot_gdf[plot_gdf['date_range'] == '2040-2069'].plot(column='mean', cmap='Reds', ax=ax2)

plot_gdf.boundary.plot(color='black', linewidth=0.1, ax=ax3)
plot_gdf[plot_gdf['date_range'] == '2070-2099'].plot(column='mean', cmap='Reds', ax=ax3)

ax1.title.set_text('Flammability by Fire Mgmt Zone: 2010-2039')
ax2.title.set_text('Flammability by Fire Mgmt Zone: 2040-2069')
ax3.title.set_text('Flammability by Fire Mgmt Zone: 2070-2099')

fig.tight_layout()
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)
ax2.axes.get_xaxis().set_visible(False)
ax2.axes.get_yaxis().set_visible(False)
ax3.axes.get_xaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)